# SAMPLE DYNAMIC PROGRAMS

In [1]:
from typing import List
# data structures
from collections import OrderedDict
import pandas as pd
# calculations
import numpy as np

# Bell numbers

https://en.wikipedia.org/wiki/Bell_number

https://www.geeksforgeeks.org/bell-numbers-number-of-ways-to-partition-a-set/

## Recursion with $S[n, k]$

In [2]:
# maximum bell number we want to calculate
N = 10

# stirling numbers of the second kind, S[n, k]
# default to 0 for convenience
snk = pd.DataFrame(0, index=range(N), columns=range(N))
# initialize using definitions
snk.loc[0, 1] = 1
snk.loc[1, 1] = 1
for n in range(2, N):
    for k in range(1, N):
        # case 0: we partition prev elt's into k-2 subsets. impossible:
        # we'll never be able to get the final cardinality to `k` !
        pass
        # case 1: uniquely determined by choice of partition of prev elt's
        # into k-1 subsets, as we then simply add last elt as singleton
        ways_to_add_last_elt_as_singleton = snk.loc[n-1, k-1]
        # case 2: we already partition the previous elt's into k subsets,
        # then choose one of those `k` to add the last elt into
        ways_to_add_last_elt_into_existing_subset = snk.loc[n-1, k] * k
        if k == n:
            assert ways_to_add_last_elt_into_existing_subset == 0, \
                f"How can {n-1} elt's be partitioned into {k} subsets??!"
        
        snk.loc[n, k] = ways_to_add_last_elt_as_singleton + \
            ways_to_add_last_elt_into_existing_subset
        del ways_to_add_last_elt_into_existing_subset
        del ways_to_add_last_elt_as_singleton
        del k
    del n
del N

bn = snk.sum(axis="columns")
del snk
print(bn)
del bn

0        1
1        1
2        2
3        5
4       15
5       52
6      203
7      877
8     4140
9    21147
dtype: int64


## Bell triangle

In [3]:
N = 10

bt = pd.DataFrame(index=range(N), columns=range(N), dtype=int)
# initialize using definition
bt.loc[0, 1] = 1
for r in range(1, N):
    bt.loc[r, 1] = bt.loc[r-1, r]
    for c in range(2, N+1):
        bt.loc[r, c] = bt.loc[r-1, c-1] + bt.loc[r, c-1]
        del c
    del r
del N

bn = bt.loc[:, 1].astype(int)
del bt
print(bn)
del bn

0        1
1        1
2        2
3        5
4       15
5       52
6      203
7      877
8     4140
9    21147
Name: 1, dtype: int64


# Subset sum

https://en.wikipedia.org/wiki/Subset_sum_problem

https://www.geeksforgeeks.org/subset-sum-problem-dp-25/

## Solution for all target sums, but "subsets" must be allowed to repeat elements

In [4]:
SET = {2, 4, 5, 12, 34}
N = len(SET)
MAX_SUM = sum(SET)

# ss[s, k] = whether we can get sum `s` from some `k`-elt subset of SET
ss = pd.DataFrame(False, index=range(0, MAX_SUM+1), columns=range(0, N+1))
# by definition, we get 0 from summing the empty (0-elt) set
ss.loc[0, 0] = True
# the only sums we can get from singletons are elt's of SET
_ss = pd.Series({s: s in SET for s in ss.index})
ss.loc[:, 1] = _ss
del _ss
# we need the answers for the `k-1`-elt subsets before
# we can answer for the `k`-elt subsets
for k in range(2, N+1):
    # given `k`, the answers for each target sum are independent,
    # so we could in theory traverse in any arbitrary order
    for s in range(1, MAX_SUM+1):
        # in order to get `s` from a k-elt subset, we must simply be able to
        # add SOME elt `x` after getting `s-x` from a `k-1`-elt subset...
        # NOTICE: THIS ALLOWS the `k-1`-subset to include `x`!
        for x in SET:
            if (s-x >= 0) and ss.loc[s-x, k-1]:
                ss.loc[s, k] = True
            del x
        del s
    del k
del MAX_SUM, N

assert not ss.loc[1:, 0].any(), \
    f"How can we get a nonzero sum from the empty set??!"
for x in SET:
    assert ss.loc[x, 1], \
        f"Trivial to get `{x}` from summing the corresponding singleton!"
    del x
del SET
print(ss.loc[[8, 9, 15, 27, 30], :])
del ss

        0      1      2      3      4      5
8   False  False   True   True   True  False
9   False  False   True   True  False  False
15  False  False  False   True   True   True
27  False  False  False  False   True   True
30  False  False  False  False   True   True


## Solution where subset must be a true subset, but single target sum must be fixed ex-ante

In [5]:
SET = sorted({2, 4, 5, 12, 34})
N = len(SET)
# hardcode: some desired sum, and whether it's possible
TGT_SUMS = [
    (0, True),  # empty set
    (1, False),
    (2, True),
    (3, False),
    (8, False),
    (9, True),
    (10, False),
    (11, True),
    (12, True),
    (13, False),
    (14, True),
    (17, True),
    (18, True),
    (19, True),
    (33, False),
    (34, True),
    (35, False),
    (100, False)
]
TGT_SUMS = OrderedDict(TGT_SUMS)
TGT_SUMS = pd.Series(TGT_SUMS)

for TGT_SUM, is_possible in TGT_SUMS.iteritems():
    # ss[n, s] = whether \exists a subset of SET[:n] s.t. sum = s
    ss = pd.DataFrame(False, index=range(N+1), columns=range(TGT_SUM+1))
    # empty set sums to zero
    ss.loc[0, 0] = True
    for n in range(1, N+1):
        for s in ss.columns:
            # remember, SET[:n] excludes the n'th elt
            last_elt = SET[n-1]
            # if last elt is too large to possibly be part of this subsol'n
            if last_elt > s:
                # the curr answer is same as prev answer for this sum
                ss.loc[n, s] = ss.loc[n-1, s]
            else:
                can_get_already = ss.loc[n-1, s]
                # if we can get up to `s-last_elt` using the previous elt's,
                # we're golden (since we can then just add last_elt)
                can_get_using_last_elt = ss.loc[n-1, s-last_elt]
                ss.loc[n, s] = can_get_already or can_get_using_last_elt
                del can_get_using_last_elt, can_get_already
            del s
        del n
    
    assert ss.loc[N, TGT_SUM] == is_possible, \
        f"{TGT_SUM} {'should' if is_possible else 'shouldnt'} be possible!"
    del ss, is_possible, TGT_SUM

del TGT_SUMS, N, SET

# Shortest path (AKA Dijkstra's problem)

https://en.wikipedia.org/wiki/Shortest_path_problem#Algorithms

https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm

In [6]:
# from the picture on the wikipedia page
NODES = [0, 1, 2, 3, 4, 5]
# distances
LINKS = {
    0: {
        1: 7,
        2: 9,
        5: 14,
    },
    1: {
        2: 10,
        3: 15,
    },
    2: {
        3: 11,
        5: 2,
    },
    3: {
        4: 6,
    },
    4: {
        5: 9,
    },
}

def lookup_node(n: int=0) -> bool:
    return n in NODES

def lookup_link(n: int=0, m: int=0) -> float:
    # self-loop
    if m == n:
        return 0
    # reorder
    n, m = min(n,m), max(n,m)
    links_from_n = LINKS.get(n, dict())
    distance_to_m = links_from_n.get(m, np.inf)
    return distance_to_m


def get_shortest_path(src: int=0, dst: int=0) -> (List[int], float):
    """Path and distance."""
    # from src to each other node
    prev = pd.Series(index=NODES, dtype=int)
    prev.loc[src] = src
    distances = pd.Series(np.inf, index=NODES)
    distances.loc[src] = 0
    unvisited = NODES.copy()
    # current node of interest
    n = src
    while n is not None:
        # consider unvisited neighbors only
        for m in unvisited:
            d_n_m = lookup_link(n=n, m=m)
            # new candidate path from src thru n to m
            d_src_n_m = distances[n] + d_n_m
            del d_n_m
            # if new candidate distance beats existing
            if d_src_n_m < distances[m]:
                prev.loc[m] = n
                distances[m] = d_src_n_m
            del d_src_n_m
            del m
        unvisited.remove(n)
        if n == dst:
            n = None
        else:
            # find shortest-path unvisited node
            n = distances[unvisited].idxmin()
    del unvisited
    
    m = dst
    path = [m]
    while m != src:
        m = prev[m]
        path.insert(0, m)
    distance = distances[dst]
    del distances, prev
    return path, distance

print(get_shortest_path(0,4))
del get_shortest_path, lookup_link, lookup_node, LINKS, NODES

([0, 2, 5, 4], 20.0)
